In [1]:
from knossos_utils import knossosdataset, skeleton
import matplotlib.pyplot as plt
import numpy as np
import neuroglancer
import os
import dxchange
%matplotlib notebook

In [2]:
# knossos_cube = '/media/Trantor2/data_raf/2018_04_03_WholeBrainMRI1_retake_newfocus/recon_flatcorr_1x/recon_crop8_new_knossos/mag1/knossos.conf'
knossos_cube = '/media/disk_trantor_2/data_hanyu/XRay/2017_10_16_VS548/subvolume_1/knossos_cube/mag1/knossos.conf'

#knossos_anno = '/media/Trantor2/data_katrina/KN_WholeBrainMRI1_skeleton&seg/annotation-20181108T1119.129.k.zip'

mag = 1
kd = knossosdataset.KnossosDataset()
kd.initialize_from_knossos_path(knossos_cube)

box_size = [512, 512, 1024]
box_offset = [ 0, 0, 0]

cube = kd.from_raw_cubes_to_matrix(
  size=box_size, 
  offset=box_offset)
#anno = kd.from_kzip_to_matrix(
#  path=knossos_anno,
#  size=box_size,
#  offset=box_offset,
#  mag=mag, 
#  verbose=False)





/home/hanyu/programs/KNOSSOS/knossos_utils/knossos_utils/knossosdataset.py:1100: UserWarning: You are using implicit channel selection. This possibility will soon be removed. Please call set_channel() before reading or writing data using KnossosDataset.
  warnings.warn('You are using implicit channel selection. This possibility will soon be removed.'


Progress: finished
Speed: 188.906 MB or MPix /s, time 1.4209973812103271


In [31]:
#ids = np.unique(anno[...])

In [3]:
print(cube.shape, cube.dtype)
#print(anno.shape, anno.dtype)



(512, 512, 1024) uint8


In [5]:
flip_cube = np.transpose(cube, [2,1,0])

In [8]:
print(cube.shape, cube.dtype)
viewer = neuroglancer.Viewer()
with viewer.txn() as s:
#   s.voxel_size = (600, 600, 600)
  s.layers['image'] = neuroglancer.ImageLayer(
    source=neuroglancer.LocalVolume(data=flip_cube, volume_type='image'))  
print(viewer.get_viewer_url())

(512, 512, 1024) uint8
http://127.0.0.1:46479/v/22b3e4bc6afd132762e8227fdd447c3e2e98b127/


Exception in callback None()
handle: <Handle cancelled>
Traceback (most recent call last):
  File "/home/hanyu/anaconda2/envs/py36/lib/python3.6/asyncio/events.py", line 145, in _run
    self._callback(*self._args)
  File "/home/hanyu/anaconda2/envs/py36/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 122, in _handle_events
    handler_func(fileobj, events)
  File "/home/hanyu/anaconda2/envs/py36/lib/python3.6/site-packages/tornado/stack_context.py", line 300, in null_wrapper
    return fn(*args, **kwargs)
  File "/home/hanyu/anaconda2/envs/py36/lib/python3.6/site-packages/tornado/iostream.py", line 713, in _handle_events
    self._handle_write()
  File "/home/hanyu/anaconda2/envs/py36/lib/python3.6/site-packages/tornado/iostream.py", line 1063, in _handle_write
    self._write_buffer.advance(num_bytes)
  File "/home/hanyu/anaconda2/envs/py36/lib/python3.6/site-packages/tornado/iostream.py", line 184, in advance
    assert 0 < size <= self._size
AssertionError
Exception 

In [7]:
del viewer

In [36]:
os.getcwd()

'/home/ravescovi/Documents/workspace/KLab-Utils/notebooks'

In [38]:

os.chdir('/media/Trantor2/Public/wholebrain_ffn')

os.getcwd()

'/media/Trantor2/Public/wholebrain_ffn'

In [39]:
dxchange.write_tiff_stack(cube,'cube/cube')

In [39]:
anno2 = anno.astype('uint8')
dxchange.write_tiff_stack(anno2,'anno/anno')

In [ ]:
skeletons = neuroglancer.skeleton.Skeleton(
  vertex_positions=np.random.randint(0,1000, size=(100,3)),
  edges=[(i,j) for i,j in zip(range(0,99),range(1,100))]
)
voxel_size = np.array([10,10,10])
class SkeletonSource(neuroglancer.skeleton.SkeletonSource):
    def __init__(self):
        super(SkeletonSource, self).__init__()
        self.vertex_attributes['affinity'] = neuroglancer.skeleton.VertexAttributeInfo(
            data_type=np.float32,
            num_components=1,
        )
        self.vertex_attributes['affinity2'] = neuroglancer.skeleton.VertexAttributeInfo(
            data_type=np.float32,
            num_components=1,
        )

    def get_skeleton(self, i):
        pos = np.unravel_index(i, shape, order='C')[::-1]
#         vertex_positions = [pos * voxel_size, pos * voxel_size + np.random.randn(3) * 1000]
        vertex_positions=np.random.randint(0,1000, size=(100,3))
        edges=[(i,j) for i,j in zip(range(0,99),range(1,100))]
        
#         edges = [0, 1]
        return neuroglancer.skeleton.Skeleton(
            vertex_positions=vertex_positions,
            edges=edges,
            vertex_attributes=dict(affinity=np.random.rand(2), affinity2=np.random.rand(2)))
